In [28]:
import os
import json
import pandas as pd
import spacy
import numpy as np
from tqdm.notebook import tqdm
from spacy.tokens import DocBin
from spacy.vocab import Vocab
from spacy.scorer import Scorer
from spacy.training import Example
from sklearn.metrics import f1_score, precision_score, recall_score

In [30]:
def _load_and_run_model(base_path, file, model):
    # Load model components and test data
    nlp = spacy.load(base_path + file + '/' + model + '/model-best')
    doc_bin = DocBin().from_disk(base_path + file + '/test.spacy')
    docs = list(doc_bin.get_docs(nlp.vocab))
    tok2vec = nlp.get_pipe('tok2vec')
    spancat = nlp.get_pipe('spancat')
    scorer = Scorer(nlp)
    
    label_rev_map = {v: k for k, v in spancat._label_map.items()}
    label_rev_map[spancat._negative_label_i] = 'no_label'

    # Run inference
    examples = []
    true_spans = []
    pred_spans = []
    for doc in docs:
        # Run pipeline
        prediction = nlp(doc.text)
        example = Example(prediction, doc)
        examples.append(example)

        # Run components separately, to access predictions for specific spans
        doc = tok2vec(doc)
        indices, scores = spancat.predict([doc])
        for span in doc.spans['sc']:
            
            # Find index to corresponding prediction
            target_value = [span.start, span.end]
            span_index = None
            for i, value in enumerate(indices.data):
                if np.array_equal(value, target_value):
                    span_index = i
                    break
                    
            # Access prediction for gold span        
            gold_span_preds = scores[[span_index]]
            predicted_label = label_rev_map[gold_span_preds.argmax()]
            
            # Store span labels for PRF calculations
            true_spans.append(span.label_)
            pred_spans.append(predicted_label)
    
    # Assess performance from inference
    scores = scorer.score(examples)
    return examples, scores, true_spans, pred_spans

def _tuple_overlap(tL, tR):
    # tL: tuple(begin, end)
    # tR: tuple(begin, end)
    tLrange = set(range(len(tL)))
    tRrange = set(range(len(tR)))
               
    InterSection = len(tLrange.intersection(tRrange))
    Union = len(tLrange.union(tRrange))

    return InterSection/Union if Union>0 else np.nan


def span_overlap_counter(examples):
    overlaplist = []
    for case in examples:
        span_set_spancat = set()
        span_set_labeled = set()
        
        for span in case.reference.spans['sc']:
            span_set_labeled.add(span)
            
        for span in case.predicted.spans['sc']:
            span_set_spancat.add(span)

        jaccard_indices = []
        for span_l in span_set_labeled:
            _jaccard_indices = []
            for span_s in span_set_spancat:
                _jaccard_indices.append(_tuple_overlap(span_l, span_s))
            try:
                max_ = max(_jaccard_indices)
            except:
                max_ = np.nan
            jaccard_indices.append(max_)
        overlaplist.append(jaccard_indices)
    return overlaplist


def span_overlap_counter_reverse(examples):
    overlaplist = []
    for case in examples:
        span_set_spancat = set()
        span_set_labeled = set()
        
        for span in case.reference.spans['sc']:
            span_set_labeled.add(span)
            
        for span in case.predicted.spans['sc']:
            span_set_spancat.add(span)

        jaccard_indices = []
        for span_s in span_set_spancat:
            _jaccard_indices = []
            for span_l in span_set_labeled:
                _jaccard_indices.append(_tuple_overlap(span_l, span_s))
            try:
                max_ = max(_jaccard_indices)
            except:
                max_ = np.nan
            jaccard_indices.append(max_)
        overlaplist.append(jaccard_indices)
    return overlaplist

In [35]:
base_path = '/training/echo/text_mining/reduced_labels/spacy_data/'

df = pd.DataFrame(columns=['entity', 'p_t_06', 'p_t_08', 'p_t_10', 'p_w_06', 'p_w_08', 'p_w_10', 'p_m_06', 'p_m_08', 'p_m_10', 
                           'r_t_06', 'r_t_08', 'r_t_10', 'r_w_06', 'r_w_08', 'r_w_10', 'r_m_06', 'r_m_08', 'r_m_10',
                           'f_t_06', 'f_t_08', 'f_t_10', 'f_w_06', 'f_w_08', 'f_w_10', 'f_m_06', 'f_m_08', 'f_m_10',
                           'jaccard_06', 'jaccard_08', 'jaccard_10', 'jaccard_rev_06', 'jaccard_rev_08', 'jaccard_rev_10',
                          'pgold_w_06', 'pgold_w_08', 'pgold_w_10', 'pgold_m_06', 'pgold_m_08', 'pgold_m_10',
                          'rgold_w_06', 'rgold_w_08', 'rgold_w_10', 'rgold_m_06', 'rgold_m_08', 'rgold_m_10',
                          'fgold_w_06', 'fgold_w_08', 'fgold_w_10', 'fgold_m_06', 'fgold_m_08', 'fgold_m_10',])

files = [x for x in os.listdir(base_path) if not x.startswith('.')]

# Iterate over all abnormalities
for file in tqdm(files):
    data = {'entity': file}

    # Iterate over model versions
    for model in [x for x in os.listdir(base_path + file) if not x.endswith('.spacy')]:
        
        # Extract model version
        nw = model.split('_')[-1]

        # Load model and data, run inference
        examples, scores, true_spans, pred_spans = _load_and_run_model(base_path, file, model)
        
        # Assess PRF for all spans
        for metric in ['p', 'r', 'f']:
            # Token-based PRF
            data[f'{metric}_t_{nw}'] = scores[f'token_{metric}']
            
            # Weighted PRF (identical to PRF reported in meta.json)
            data[f'{metric}_w_{nw}'] = scores[f'spans_sc_{metric}']
            
            # Macro PRF
            data[f'{metric}_m_{nw}'] = np.mean([v[f'{metric}'] for _, v in scores['spans_sc_per_type'].items()])    

        # Assess Jaccard index
        OverlapJaccardIndices = span_overlap_counter(examples)
        OverlapJaccardIndicesRev = span_overlap_counter_reverse(examples)
        data[f'jaccard_{nw}'] = np.nanmean([_v for v in OverlapJaccardIndices for _v in v])
        data[f'jaccard_rev_{nw}'] = np.nanmean([_v for v in OverlapJaccardIndicesRev for _v in v])

        # Weighted PRF for gold spans
        data[f'pgold_w_{nw}'] = precision_score(true_spans, pred_spans, average='weighted', zero_division=0)
        data[f'rgold_w_{nw}'] = recall_score(true_spans, pred_spans, average='weighted', zero_division=0)
        data[f'fgold_w_{nw}'] = f1_score(true_spans, pred_spans, average='weighted', zero_division=0)

        # Macro PRF for gold spans
        data[f'pgold_m_{nw}'] = precision_score(true_spans, pred_spans, average='macro', zero_division=0)
        data[f'rgold_m_{nw}'] = recall_score(true_spans, pred_spans, average='macro', zero_division=0)
        data[f'fgold_m_{nw}'] = f1_score(true_spans, pred_spans, average='macro', zero_division=0)
        
    # Add data row
    df.loc[len(df)] = data

  0%|          | 0/11 [00:00<?, ?it/s]

In [36]:
# Choose entity-specific best model with regards to negative weight
# Based on highest F-score
df['f_w_max'] = np.nanmax(df[['f_w_06', 'f_w_08', 'f_w_10']], axis=1)
df['nw_max'] = np.nanargmax(df[['f_w_06', 'f_w_08', 'f_w_10']], axis=1)
df['nw_max'].replace({0: '06', 1: '08', 2: '10'}, inplace=True)

# Pick corresponding performance metrics
df['jaccard_max'] = df.apply(lambda x: round(x['jaccard_' + str(x['nw_max'])], 2), axis=1)
df['jaccard_rev_max'] = df.apply(lambda x: round(x['jaccard_rev_' + str(x['nw_max'])], 2), axis=1)

for metric in ['p', 'r', 'f']:
    df[f'{metric}'] = df.apply(lambda x: str(round(x[f'{metric}_w_' + str(x['nw_max'])], 2)) + ' (' + str(round(x[f'{metric}_m_' + str(x['nw_max'])], 2)) + ')', axis=1)
    df[f'{metric}gold'] = df.apply(lambda x: str(round(x[f'{metric}gold_w_' + str(x['nw_max'])], 2)) + ' (' + str(round(x[f'{metric}gold_m_' + str(x['nw_max'])], 2)) + ')', axis=1)

In [37]:
df.sort_values('entity', inplace=True).reset_index(drop=True)

In [38]:
df

,entity,p_t_06,p_t_08,p_t_10,p_w_06,p_w_08,p_w_10,p_m_06,p_m_08,p_m_10,...,f_w_max,nw_max,jaccard_max,jaccard_rev_max,p,pgold,r,rgold,f,fgold
3,aortic_regurgitation,0.919592,0.919592,0.919592,0.953162,0.969072,0.961832,0.726851,0.487992,0.481824,...,0.888646,06,0.97,0.98,0.95 (0.73),0.97 (0.58),0.83 (0.59),0.83 (0.47),0.89 (0.65),0.9 (0.52)
1,aortic_stenosis,0.918409,0.918409,0.918409,0.889286,0.884211,0.901460,0.808844,0.830746,0.895150,...,0.818182,08,0.95,0.97,0.88 (0.83),0.99 (0.76),0.76 (0.66),0.76 (0.51),0.82 (0.73),0.86 (0.6)
10,diastolic_dysfunction,0.918409,0.918409,0.918409,0.925490,0.906015,0.926923,0.884612,0.864572,0.922596,...,0.874773,10,0.95,0.96,0.93 (0.92),0.98 (0.77),0.83 (0.77),0.82 (0.61),0.87 (0.83),0.89 (0.68)
4,lv_dil,0.918409,0.918409,0.918409,0.813679,0.841232,0.834606,0.460690,0.694208,0.528561,...,0.817972,08,0.92,0.93,0.84 (0.69),0.96 (0.59),0.8 (0.68),0.78 (0.57),0.82 (0.68),0.86 (0.57)
5,lv_syst_func,0.918409,0.918409,0.918409,0.767179,0.775424,0.760000,0.367394,0.372593,0.374272,...,0.754032,06,0.90,0.90,0.77 (0.37),0.93 (0.43),0.74 (0.36),0.68 (0.28),0.75 (0.36),0.78 (0.33)
6,mitral_regurgitation,0.918409,0.918409,0.918409,0.949791,0.946612,0.966140,0.703604,0.716748,0.712843,...,0.921079,08,0.97,0.97,0.95 (0.72),0.95 (0.58),0.9 (0.69),0.88 (0.52),0.92 (0.7),0.91 (0.55)
2,pe,0.925300,0.925300,0.925300,0.833333,0.903743,0.891192,0.266836,0.313543,0.306222,...,0.787185,10,0.96,0.97,0.89 (0.31),0.84 (0.3),0.7 (0.26),0.7 (0.21),0.79 (0.28),0.76 (0.24)
0,rv_dil,0.924163,0.924163,0.924163,0.937500,0.916155,0.913934,0.751793,0.692235,0.724523,...,0.881890,08,0.99,0.99,0.92 (0.69),0.97 (0.62),0.85 (0.67),0.84 (0.55),0.88 (0.68),0.9 (0.58)
9,rv_syst_func,0.918409,0.918409,0.918409,0.892857,0.926437,0.899358,0.694759,0.524931,0.714256,...,0.881743,06,0.96,0.97,0.89 (0.69),0.94 (0.62),0.87 (0.61),0.85 (0.51),0.88 (0.63),0.89 (0.55)
8,tricuspid_regurgitation,0.918409,0.918409,0.918409,0.925072,0.932515,0.929448,0.858377,0.850428,0.837897,...,0.906780,06,0.97,0.98,0.93 (0.86),0.96 (0.72),0.89 (0.81),0.88 (0.63),0.91 (0.83),0.92 (0.67)


In [42]:
## Table 3 - PRF scores for the exact gold spans
#df[['entity', 'fgold', 'rgold', 'pgold']].to_latex('/training/echo/text_mining/output/table3_pipeline_performance_goldspans.tex', index=False)
df[['entity', 'fgold', 'rgold', 'pgold']]

,entity,fgold,rgold,pgold
3,aortic_regurgitation,0.9 (0.52),0.83 (0.47),0.97 (0.58)
1,aortic_stenosis,0.86 (0.6),0.76 (0.51),0.99 (0.76)
10,diastolic_dysfunction,0.89 (0.68),0.82 (0.61),0.98 (0.77)
4,lv_dil,0.86 (0.57),0.78 (0.57),0.96 (0.59)
5,lv_syst_func,0.78 (0.33),0.68 (0.28),0.93 (0.43)
6,mitral_regurgitation,0.91 (0.55),0.88 (0.52),0.95 (0.58)
2,pe,0.76 (0.24),0.7 (0.21),0.84 (0.3)
0,rv_dil,0.9 (0.58),0.84 (0.55),0.97 (0.62)
9,rv_syst_func,0.89 (0.55),0.85 (0.51),0.94 (0.62)
8,tricuspid_regurgitation,0.92 (0.67),0.88 (0.63),0.96 (0.72)


In [40]:
## Table 4 - PRF scores for all predicted spans
#df[['entity', 'f', 'r', 'p']].to_latex('/training/echo/text_mining/output/table4_pipeline_performance.tex', index=False)
df[['entity', 'f', 'r', 'p']]

,entity,f,r,p
0,aortic_regurgitation,0.89 (0.65),0.83 (0.59),0.95 (0.73)
1,aortic_stenosis,0.82 (0.73),0.76 (0.66),0.88 (0.83)
2,diastolic_dysfunction,0.87 (0.83),0.83 (0.77),0.93 (0.92)
3,lv_dil,0.82 (0.68),0.8 (0.68),0.84 (0.69)
4,lv_syst_func,0.75 (0.36),0.74 (0.36),0.77 (0.37)
5,mitral_regurgitation,0.92 (0.7),0.9 (0.69),0.95 (0.72)
6,pe,0.79 (0.28),0.7 (0.26),0.89 (0.31)
7,rv_dil,0.88 (0.68),0.85 (0.67),0.92 (0.69)
8,rv_syst_func,0.88 (0.63),0.87 (0.61),0.89 (0.69)
9,tricuspid_regurgitation,0.91 (0.83),0.89 (0.81),0.93 (0.86)


In [41]:
## Table 5 - Jaccard similarity for all gold spans and for all predicted spans 
#df[['entity', 'jaccard_max', 'jaccard_rev_max']].to_latex('/training/echo/text_mining/output/table5_jaccard_labeltospan.tex', index=False)
df[['entity', 'jaccard_max', 'jaccard_rev_max']]

,entity,jaccard_max,jaccard_rev_max
0,aortic_regurgitation,0.97,0.98
1,aortic_stenosis,0.95,0.97
2,diastolic_dysfunction,0.95,0.96
3,lv_dil,0.92,0.93
4,lv_syst_func,0.90,0.90
5,mitral_regurgitation,0.97,0.97
6,pe,0.96,0.97
7,rv_dil,0.99,0.99
8,rv_syst_func,0.96,0.97
9,tricuspid_regurgitation,0.97,0.98
